In [34]:
import numpy as np
import pandas as pd

In [35]:
file = 'C:\\Users\\reill\\Downloads\\Traffic_Crashes_-_Crashes.csv'
df = pd.DataFrame(pd.read_csv(file))

In [36]:
#Drop high NaN count columns
dropcols = []
for col in df:
    if (df[col].isna().sum() / len(df[col])) > 0.6:
        dropcols.append(col)
df.drop(dropcols, axis='columns', inplace=True)

#Drop unnecessary columns
df.drop(['CRASH_RECORD_ID', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED', 'LOCATION', 'CRASH_DATE'], axis='columns', inplace=True)
df.replace('UNKNOWN', np.nan, inplace=True)

df = df.dropna(axis='index').reset_index(drop=True)

In [37]:
categorical = ['TRAFFIC_CONTROL_DEVICE',
       'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
       'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'ALIGNMENT',
       'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE',
       'SEC_CONTRIBUTORY_CAUSE', 'MOST_SEVERE_INJURY']
#Removed street name and direction

In [38]:
numerical = ['POSTED_SPEED_LIMIT', 'STREET_NO', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE']

#This leaves out the datetime column

In [39]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = df
scaled_df[numerical] = scaler.fit_transform(scaled_df[numerical])
scaled_df.head()

,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
0,0.252263,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,1.0,0.0,0.155627,1.457387,0.062053,0.001778,0.010912
1,0.252263,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,4.0,0.0,-0.572321,-1.586060,0.357042,0.355384,-0.050762
2,0.252263,TRAFFIC SIGNAL,FUNCTIONING IMPROPERLY,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,5.0,0.0,0.0,0.0,1.065562,-0.064337,0.357042,-0.276717,0.067092
3,0.252263,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,2.0,0.0,0.883575,-0.064337,0.652031,0.198169,0.014718
4,0.252263,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE OPPOSITE DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,2.0,0.0,0.883575,-1.586060,0.062053,0.116960,-0.055930


In [40]:
scaled_df = pd.get_dummies(scaled_df)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [42]:
mini = scaled_df.sample(60000, axis='index')

x = mini.drop(['INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN'], axis='columns')
y = mini['INJURIES_FATAL']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [43]:
forest_fatal = RandomForestRegressor(n_jobs=16)
forest_fatal.fit(x_train, y_train)

RandomForestRegressor(n_jobs=16)

In [44]:
y_pred = forest_fatal.predict(x_test)

from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9282427321269653

In [45]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
pca.fit(x)

PCA(n_components=300)

In [46]:
x = pd.DataFrame(pca.transform(x))

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

forest_fatal = RandomForestRegressor(n_jobs=16)
forest_fatal.fit(x_train, y_train)

y_pred = forest_fatal.predict(x_test)
r2_score(y_test, y_pred)

0.9128038277511963

In [49]:
pd.Series(y_pred).value_counts()

0.00    11977
1.01        4
0.05        3
0.07        2
1.04        2
1.03        1
0.10        1
0.02        1
1.00        1
0.61        1
1.02        1
1.23        1
1.24        1
0.11        1
0.43        1
1.18        1
1.19        1
dtype: int64

## So the model is basically just guessing zero every time and happens to be right in most cases